In [ ]:
import pandas as pd
import numpy as np


In [1]:
df_m = pd.read_csv("movies.csv")
df_r = pd.read_csv("ratings.csv")

NameError: ignored

In [ ]:
#посмотреть, что получилось
df_r.head()

,user id,item id,ratings,timestamp,Date,Month,Year
0,1,1,5,874965758,22.09.1997,9,1997
1,1,10,3,875693118,01.10.1997,10,1997
2,1,100,5,878543541,03.11.1997,11,1997
3,1,101,2,878542845,03.11.1997,11,1997
4,1,102,2,889751736,13.03.1998,3,1998


In [ ]:
#timestamp нам вроде ни к чему, нигде далее он не употребляется. 
#Месяц и год тоже были вспомогательными, убираем
df_r=df_r[['user id', 'item id', 'ratings', 'Date']]

In [ ]:
#считаем количество повторений по столбцу
m=df_r['user id'].value_counts().reset_index()
#нахожу ячейку, где максимум 
m=m.loc[m['user id']== m['user id'].max()]

print(m['index'])


0    405
Name: index, dtype: int64


In [ ]:
t=m['index']   #вот это число 405 и есть номер юзера. Я не понимаю, как его дальше
#передать в виде переменной, а не числом
df_405=df_r.loc[df_r['user id'] == 405]
df_405

,user id,item id,ratings,Date
37144,405,1004,1,23.01.1998
37145,405,1005,1,23.01.1998
37146,405,1006,1,23.01.1998
37147,405,101,1,23.01.1998
37148,405,1018,1,23.01.1998
...,...,...,...,...
37876,405,99,5,23.01.1998
37877,405,994,1,23.01.1998
37878,405,996,1,23.01.1998
37879,405,997,1,23.01.1998


In [ ]:
#сливаем две таблицы по одинаковым столбцам. Как их убрать и оставить один? Тоже не поняла...
df=pd.merge(df_r,df_m,  left_on=('item id'), right_on=('movie id'), how="inner") 
df=df.drop(columns=['IMDb URL','Unnamed: 3'])
df

,user id,item id,ratings,Date,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,5,22.09.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10,1,4,26.10.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,101,1,3,18.10.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,102,1,3,02.01.1998,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,106,1,4,06.12.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,07.03.1998,1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
99996,863,1680,2,07.03.1998,1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
99997,896,1681,3,11.02.1998,1681,You So Crazy (1994),01-Jan-1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
99998,90,1201,5,31.03.1998,1201,Marlene Dietrich: Shadow and Light (1996),02-Apr-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#количество оценок от всех пользователей на фильм
df_group=df.groupby('movie id')['ratings'].agg([np.sum,np.size])#agg({'user id': 'count','ratings': 'sum'})
df_group


,sum,size
movie id,,
1,1753,452
2,420,131
3,273,90
4,742,209
5,284,86
...,...,...
1678,1,1
1679,3,1
1680,2,1


In [ ]:
#слили две таблицы по столбцу муви ид
df=df.merge(df_group,  left_on=('movie id'), right_on=('movie id'), how="inner") 
df

,user id,item id,ratings,Date,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,sum,size
0,1,1,5,22.09.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1753,452
1,10,1,4,26.10.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1753,452
2,101,1,3,18.10.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1753,452
3,102,1,3,02.01.1998,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1753,452
4,106,1,4,06.12.1997,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1753,452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,07.03.1998,1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,3,1
99996,863,1680,2,07.03.1998,1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,2,1
99997,896,1681,3,11.02.1998,1681,You So Crazy (1994),01-Jan-1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
99998,90,1201,5,31.03.1998,1201,Marlene Dietrich: Shadow and Light (1996),02-Apr-1996,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1


In [ ]:
df.columns

Index(['user id', 'item id', 'ratings', 'Date', 'movie id', 'movie title',
       'release date', 'unknown', 'Action', 'Adventure', 'Animation',
       'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [ ]:
#я решила, что метриками и являются посчитанные ранее общее количество оценок и сумма рейтинга
#А  предсказать можно пытаться жанр. Т.е. предпочитаемый большинством жанр фильма. 
#Честно говоря, не понимаю, что еще можно тут предсказывать, не номер же в базе и не название.
x, y = df[['movie id','sum','size']],df[['Action','Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi','Thriller', 'War', 'Western']]
y.head()

,Action,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2)
x_train.head()

,movie id,sum,size
92657,775,72,26
20828,215,784,212
340,1,1753,452
11554,172,1543,367
26697,248,588,160


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()

In [ ]:
model.fit(x_train, y_train)
model_prediction=model.predict(x_train)


In [ ]:
print("Предсказание:", model_prediction)

Предсказание: [[0.11774049 0.33380833 0.05453137 ... 0.16366648 0.02375476 0.02216123]
 [0.30896137 0.30377016 0.09330301 ... 0.24277159 0.10887516 0.01967968]
 [0.50798556 0.18211102 0.12073038 ... 0.31493093 0.2562801  0.00685225]
 ...
 [0.36563615 0.30705568 0.07086781 ... 0.28009329 0.11743197 0.01233283]
 [0.45122916 0.36134771 0.07449381 ... 0.33133409 0.10572456 0.01365206]
 [0.17171659 0.36877457 0.04567725 ... 0.20000374 0.01314175 0.02055603]]


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
model_mse = mean_squared_error(model_prediction, y_train)
print("Computed error:", model_mse)

Computed error: 0.09215582218023485


In [ ]:
model.fit(x_test, y_test)
model_prediction=model.predict(x_test)

model_mse2 = mean_squared_error(model_prediction, y_test)
print("Computed error2:", model_mse2)

Computed error2: 0.09161050619827954


In [ ]:
df.to_csv ('movie_merge.csv', index = False, header=True)